In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Activation
from tensorflow.keras.models import Sequential
import os
from keras.optimizers import Adam,SGD
import tensorflow as tf

In [3]:
train_dir="Emotion/train"
test_dir="Emotion/test"


In [4]:
train_datagen=ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=30,
                                shear_range= 0.3,
                                zoom_range= 0.3,
                                horizontal_flip=False,
                                fill_mode="nearest"
                                )
test_datagen=ImageDataGenerator(
                                rescale=1./255,
                                rotation_range=30,
                                shear_range= 0.3,
                                zoom_range= 0.3,
                                horizontal_flip=False,
                                fill_mode="nearest"
                                )

In [5]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=512,
    class_mode='categorical',
    shuffle=True
)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=512,
    class_mode='categorical',
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
filepath = os.path.join("./mymodels/mymodel.hdf5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')
callbacks = [checkpoint]

In [7]:
class_labels=['angry','disgusted','fearful','happy','neutral','sad','surprised']

In [8]:
img,labels=train_generator.__next__()


In [9]:
model=Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',input_shape=(48,48,1)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

#2nd CNN layer
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#3rd CNN
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
#fully connected First layer 
model.add(Dense(64))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(7,activation='softmax'))
#model.add(layers.)
#model.add(layers.)
#model.add(layers.)

In [10]:
opt=Adam(learning_rate=0.001)

In [11]:
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=['accuracy']
    )

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 activation_1 (Activation)   (None, 16, 16, 64)        0         
                                                        

In [13]:
num_train_images=0
for root,dirs,files in os.walk(train_dir):
    num_train_images+=len(files)
    
num_test_images=0
for root,dirs,files in os.walk(test_dir):
    num_test_images+=len(files)
num_train_images,num_test_images

(28709, 7178)

In [14]:
epochs=2

In [15]:
trained_model=model.fit_generator(train_generator,
                    steps_per_epoch=num_train_images//512,
                    epochs=epochs,
                    validation_data = test_generator,
                    validation_steps = num_test_images//512,
                    callbacks=callbacks
                    )

C:\Users\sarve\AppData\Local\Temp\ipykernel_3844\270675021.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  trained_model=model.fit_generator(train_generator,


Epoch 1/2
36/56 [==================>...........] - ETA: 50s - loss: 1.8538 - accuracy: 0.2255

KeyboardInterrupt: 